In [1]:
import boto3, re
import sagemaker
from sagemaker import get_execution_role
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import zipfile
import copy
import time
import anomaly_TFconversion
from sagemaker.predictor import csv_serializer, json_deserializer

sagemaker_session = sagemaker.Session()
role = get_execution_role()
bucket='sagemaker-tensorflow-anomaly-detection'
prefix='Anomaly'

In [2]:
data_All=pd.read_csv('Drill-Data-All.csv')
data_All=pd.DataFrame(data_All)
data_All.drop(data_All.columns[[5, 6, 7,8,9,10,11,12,13,14,15,16]], axis = 1, inplace = True) 
data=data_All
print(data)

      HasAnomaly  AvgPressure     AvgMotor   AvgSpindle   duration
0              1    24.343573  1431.292714   562.840922  20.035540
1              0    10.494685  1434.177556  1412.971635  13.020580
2              0    10.473129  1425.126240  1414.247483  11.017775
3              1    24.253581  1428.827567  1411.949713  20.027548
4              0    10.501866  1431.960981  1412.700847  12.020303
...          ...          ...          ...          ...        ...
1836           0    10.498714  1423.956407  1404.029905  13.203029
1837           0    10.526026  1424.240970  1409.382090  14.063524
1838           1    24.242122   857.497394  1412.478031  14.550770
1839           1    10.507492  2212.217378  1413.583380  21.052171
1840           0    10.476097  1424.493528  1405.768496  10.514672

[1841 rows x 5 columns]


In [3]:
vector_data=data
TRAIN_ATTRIBUTES=['AvgPressure','AvgMotor','AvgSpindle','duration']
def normalizeVectorized(data, attributes=TRAIN_ATTRIBUTES, description=None):
    if (description is None):
        description = data.describe()
    # make a deep copy to avoid reference bugs
    resultData = copy.deepcopy(data)
    # normalize by attribute
    for attr in attributes:
        min_val = float(description[attr]["min"])
        max_val = float(description[attr]["max"])
        
        # vectorized normalization implementation
        resultData[attr] = (resultData[attr] - min_val) / (max_val - min_val)
    return resultData

vectorizedResult = normalizeVectorized(vector_data)
print(vectorizedResult)

      HasAnomaly  AvgPressure  AvgMotor  AvgSpindle  duration
0              1     0.990762  0.423631    0.007349  0.793192
1              0     0.160610  0.425730    0.536238  0.238155
2              0     0.159318  0.419144    0.537031  0.079690
3              1     0.985368  0.421837    0.535602  0.792559
4              0     0.161041  0.424117    0.536069  0.159012
...          ...          ...       ...         ...       ...
1836           0     0.160852  0.418293    0.530675  0.252591
1837           0     0.162489  0.418500    0.534005  0.320675
1838           1     0.984681  0.006123    0.535931  0.359227
1839           1     0.161378  0.991851    0.536618  0.873629
1840           0     0.159496  0.418684    0.531756  0.039883

[1841 rows x 5 columns]


In [4]:
data_copy = vectorizedResult.copy()
train_set = data_copy.sample(frac=0.75, random_state=0)
eval_set = data_copy.sample(frac=0.20, random_state=0)
test_set = data_copy.sample(frac=0.05, random_state=0)
test_set.to_csv('test_set.csv')

In [5]:
train_df=pd.DataFrame(train_set)
test_df=pd.DataFrame(eval_set)
dataset=anomaly_TFconversion.data_preprocess(train_df, test_df)
print(dataset["train_dict"])

{'features': array([[0.16030103, 0.41915555, 0.53432185, 0.15895943],
       [0.16082714, 0.41911651, 0.53357234, 0.04080207],
       [0.16059293, 0.42083073, 0.53238253, 0.19831018],
       ...,
       [0.16012353, 0.42652581, 0.53845574, 0.05661203],
       [0.16090419, 0.42326217, 0.53222657, 0.39657396],
       [0.00127664, 0.4180939 , 0.        , 0.37653673]]), 'labels': array([0, 0, 0, ..., 0, 0, 1])}


In [6]:
!mkdir 'tfrecord_data'
anomaly_TFconversion.convert_to(dataset["train_dict"], 'train', 'tfrecord_data')
anomaly_TFconversion.convert_to(dataset["test_dict"], 'test', 'tfrecord_data')

mkdir: cannot create directory ‘tfrecord_data’: File exists
Writing tfrecord_data/train.tfrecords

Writing tfrecord_data/test.tfrecords


In [7]:
inputs = sagemaker_session.upload_data(path='tfrecord_data',bucket=bucket, key_prefix='data/Anomaly_Tfrecord')

In [8]:
from sagemaker.tensorflow import TensorFlow
output_path = "s3://sagemaker-tensorflow-anomaly-detection/Anomaly"

anomaly_estimator = TensorFlow(entry_point='anomaly_classifier.py',
                             role=role,
                             framework_version='1.11.0',
                             output_path = output_path,
                             training_steps=1000, 
                             evaluation_steps=100,
                             #early_stopping="true",
                             #patience="5",
                             train_instance_count=1,
                             train_instance_type='ml.m4.xlarge')

anomaly_estimator.fit(inputs)

2.1.0 is the latest version of tensorflow that supports Python 2. Newer versions of tensorflow will only be available for Python 3.Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-10-14 11:14:29 Starting - Starting the training job...
2020-10-14 11:14:30 Starting - Launching requested ML instances......
2020-10-14 11:15:32 Starting - Preparing the instances for training...
2020-10-14 11:16:28 Downloading - Downloading input data
2020-10-14 11:16:28 Training - Downloading the training image...
2020-10-14 11:16:47 Training - Training image download completed. Training in progress.2020-10-14 11:16:48,003 INFO - root - running container entrypoint
2020-10-14 11:16:48,004 INFO - root - starting train task
2020-10-14 11:16:48,019 INFO - container_support.training - Training starting
2020-10-14 11:16:50,228 INFO - tf_container - ----------------------TF_CONFIG--------------------------
2020-10-14 11:16:50,228 INFO - tf_container - {"environment": "cloud", "cluster": {"master": ["algo-1:2222"]}, "task": {"index": 0, "type": "master"}}
2020-10-14 11:16:50,228 INFO - tf_container - ---------------------------------------------------------
2020-10-14 11:16:50,228 INFO

2020-10-14 11:16:56,521 INFO - tensorflow - Calling model_fn.
2020-10-14 11:16:56,641 INFO - tensorflow - Done calling model_fn.
2020-10-14 11:16:56,661 INFO - tensorflow - Starting evaluation at 2020-10-14-11:16:56
2020-10-14 11:16:56,804 INFO - tensorflow - Graph was finalized.
2020-10-14 11:16:57,129 INFO - tensorflow - Restoring parameters from s3://sagemaker-tensorflow-anomaly-detection/Anomaly/sagemaker-tensorflow-2020-10-14-11-14-28-884/checkpoints/model.ckpt-1000
2020-10-14 11:16:57,297 INFO - tensorflow - Running local_init_op.
2020-10-14 11:16:57,307 INFO - tensorflow - Done running local_init_op.
2020-10-14 11:16:57,407 INFO - tensorflow - Evaluation [10/100]
2020-10-14 11:16:57,422 INFO - tensorflow - Evaluation [20/100]
2020-10-14 11:16:57,436 INFO - tensorflow - Evaluation [30/100]
2020-10-14 11:16:57,451 INFO - tensorflow - Evaluation [40/100]
2020-10-14 11:16:57,465 INFO - tensorflow - Evaluation [50/100]
2020-10-14 11:16:57,481 INFO - tensorflow - Evaluation [60/100]
2


2020-10-14 11:17:10 Uploading - Uploading generated training model
2020-10-14 11:17:10 Completed - Training job completed
Training seconds: 50
Billable seconds: 50


In [9]:
# output_path = '/'.join(anomaly_estimator.output_path.split('/')[:-1])

# print(output_path)


# optimized_estimator = anomaly_estimator.compile_model(target_instance_family='rasp3b',
#                               output_path=output_path,
#                               input_shape= {'data':[1,4]},
#                               framework='tensorflow', framework_version='1.11.0')

In [10]:
anomaly_predictor = anomaly_estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
2.1.0 is the latest version of tensorflow that supports Python 2. Newer versions of tensorflow will only be available for Python 3.Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------!

In [11]:
anomaly_predictor.content_type = 'text/csv'
anomaly_predictor.serializer = csv_serializer
anomaly_predictor.deserializer = json_deserializer

In [12]:
test=pd.read_csv('test_set.csv',index_col= 0)
test1=test
test1=pd.DataFrame(test1)
test_label=test1['HasAnomaly']
test_label=np.array(test_label.values)
print(test_label)
test=test.drop(columns='HasAnomaly')
test_feature_dataset = test[['AvgPressure','AvgMotor','AvgSpindle','duration']]
test_features = np.array(test_feature_dataset.values).astype('float32')
print(test_features)

[0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 1 1
 0 0 1 1 0 0 1 0 1 0 1 0 1 1 1 0 0 1 1 1 0 1 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0
 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0]
[[1.60301030e-01 4.19155538e-01 5.34321845e-01 1.58959433e-01]
 [1.60827145e-01 4.19116527e-01 5.33572316e-01 4.08020727e-02]
 [1.60592929e-01 4.20830727e-01 5.32382548e-01 1.98310181e-01]
 [1.60663113e-01 4.25064236e-01 5.32809794e-01 1.66578099e-01]
 [1.62973240e-01 4.25353855e-01 5.43324649e-01 3.97964083e-02]
 [1.62636340e-01 9.85588431e-01 8.97760876e-03 3.96756947e-01]
 [1.61942139e-01 4.27221060e-01 5.31268716e-01 1.98335230e-01]
 [1.61192700e-01 4.19254422e-01 5.29519320e-01 1.98041022e-01]
 [1.63517460e-01 4.25673634e-01 5.37286937e-01 2.77588338e-01]
 [1.59699544e-01 4.21278507e-01 5.35318255e-01 4.01282087e-02]
 [2.90872296e-03 4.19079512e-01 6.60462910e-03 3.37259769e-01]
 [9.79002297e-01 2.63638096e-03 5.36523342e-01 1.98898464e-01]
 [1.58487275e-01 4.19918150e-01 5.28104186e-01 3.9681890

In [13]:
# from numpy import asarray
# from numpy import save
# data = asarray(test)
# save('data.npy', data)

In [14]:
true_pred = 0
false_pred = 0
startTime = time.time()
for tf in range(len(test_features)):
    prediction = anomaly_predictor.predict(test_features[tf])
    #print(prediction['outputs']['class_ids']['int64Val'][0])
    if(test_label[tf]==int(prediction['outputs']['class_ids']['int64Val'][0])):
        true_pred +=1
    else:
        false_pred+=1

endTime = time.time()
difference=endTime-startTime
print(difference)
print(true_pred,false_pred)
print((true_pred/len(test_features))*100)

1.4853756427764893
89 3
96.73913043478261


In [15]:
anomaly_predictor.delete_endpoint()